In [2]:
import cv2
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os

In [34]:
with open('dataset_blindfolded.npy', 'rb') as f:
    inp_mat= np.load(f)
    out_mat = np.load(f)

In [35]:
inp_mat, out_mat = sklearn.utils.shuffle(inp_mat, out_mat)

In [36]:
inp_mat.shape

(2832752, 7, 7)

In [37]:
out_mat.shape

(2832752,)

In [38]:
action=['left','right','up','down', 'stop']

In [39]:
out_mat=tf.keras.utils.to_categorical( out_mat, 5 )

In [40]:
x_train,y_train,x_test,y_test=inp_mat[:1832752],out_mat[:1832752],inp_mat[1832752:],out_mat[1832752:]

In [11]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1832752, 7, 7), (1832752, 5), (1000000, 7, 7), (1000000, 5))

# CNN starts here

In [12]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=[7, 7,1]))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
#cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
#cnn.add(tf.keras.layers.Dropout(0.25))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=5, activation='softmax'))
cnn.compile(optimizer = 'adam', loss ='categorical_crossentropy' , metrics = ['accuracy'])

In [13]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 5, 5, 64)          640       
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 1, 64)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 5)                 645       
                                                        

In [14]:
cnn.fit( x_train, y_train, epochs = 10 ,validation_data=(x_test, y_test))

Epoch 1/10
57274/57274 [==============================] - 78s 1ms/step - loss: 0.4084 - accuracy: 0.8281 - val_loss: 0.3739 - val_accuracy: 0.8424
Epoch 2/10
57274/57274 [==============================] - 75s 1ms/step - loss: 0.3673 - accuracy: 0.8457 - val_loss: 0.3617 - val_accuracy: 0.8487
Epoch 3/10
57274/57274 [==============================] - 75s 1ms/step - loss: 0.3587 - accuracy: 0.8497 - val_loss: 0.3585 - val_accuracy: 0.8499
Epoch 4/10
57274/57274 [==============================] - 75s 1ms/step - loss: 0.3542 - accuracy: 0.8515 - val_loss: 0.3525 - val_accuracy: 0.8524
Epoch 5/10
57274/57274 [==============================] - 76s 1ms/step - loss: 0.3511 - accuracy: 0.8530 - val_loss: 0.3557 - val_accuracy: 0.8504
Epoch 6/10
57274/57274 [==============================] - 76s 1ms/step - loss: 0.3494 - accuracy: 0.8534 - val_loss: 0.3529 - val_accuracy: 0.8528
Epoch 7/10
57274/57274 [==============================] - 75s 1ms/step - loss: 0.3477 - accuracy: 0.8543 - val_loss: 0

In [134]:
import json
model_json = cnn.to_json()
with open("cnn_model_normal_astar.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn.save_weights("cnn_model_normal_astar.h5")
print("Saved model to disk")

Saved model to disk


# ANN Starts here

In [41]:
# change shape based on window grid size
digit_input = tf.keras.layers.Input( shape = (7,7) )
flatten_image = tf.keras.layers.Flatten()( digit_input )
dense_1 = tf.keras.layers.Dense( units = 128, activation = tf.nn.relu )( flatten_image )
dense_2 = tf.keras.layers.Dense( units = 64, activation = tf.nn.relu )( dense_1 )
dense_3 = tf.keras.layers.Dense( units = 64, activation = tf.nn.relu )( dense_2 )
dense_4=tf.keras.layers.Dense( units = 32,activation = tf.nn.relu )( dense_3)
logits = tf.keras.layers.Dense( units = 5, activation = None )( dense_4 )
probabilities = tf.keras.layers.Softmax()( logits )
ann = tf.keras.Model( inputs = digit_input, outputs = probabilities )
ann.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [48]:
history = ann.fit( x_train, y_train, epochs = 10 ,validation_data=(x_test, y_test))

Epoch 1/10
57274/57274 [==============================] - 40s 699us/step - loss: 0.3927 - accuracy: 0.8311 - val_loss: 0.3608 - val_accuracy: 0.8463
Epoch 2/10
57274/57274 [==============================] - 40s 694us/step - loss: 0.3481 - accuracy: 0.8516 - val_loss: 0.3409 - val_accuracy: 0.8544
Epoch 3/10
57274/57274 [==============================] - 39s 680us/step - loss: 0.3379 - accuracy: 0.8566 - val_loss: 0.3420 - val_accuracy: 0.8544
Epoch 4/10
57274/57274 [==============================] - 39s 683us/step - loss: 0.3323 - accuracy: 0.8595 - val_loss: 0.3352 - val_accuracy: 0.8580
Epoch 5/10
57274/57274 [==============================] - 38s 670us/step - loss: 0.3292 - accuracy: 0.8610 - val_loss: 0.3307 - val_accuracy: 0.8603
Epoch 6/10
57274/57274 [==============================] - 39s 686us/step - loss: 0.3270 - accuracy: 0.8620 - val_loss: 0.3293 - val_accuracy: 0.8612
Epoch 7/10
57274/57274 [==============================] - 39s 682us/step - loss: 0.3255 - accuracy: 0.8629

In [49]:
ann.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 7, 7)]            0         
                                                                 
 flatten_1 (Flatten)         (None, 49)                0         
                                                                 
 dense_2 (Dense)             (None, 128)               6400      
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 5)                 165   

In [33]:
# SAVE MODEL
import json
model_json = ann.to_json()
with open("ann_model_blindfolded.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
ann.save_weights("ann_model_blindfolded.h5")
print("Saved model to disk")




Saved model to disk


In [27]:
# load previously saved model
from keras.models import model_from_json

json_file = open('ann_model_blindfolded.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
ann = model_from_json(loaded_model_json)
# load weights into new model
ann.load_weights("ann_model_blindfolded.h5")
print("Loaded model from disk")

Loaded model from disk


In [28]:
import math
import time
from heapq import heappush, heappop

import numpy as np


class PriorityQueue:

    def __init__(self, iterable=[]):
        self.heap = []
        for value in iterable:
            heappush(self.heap, (0, value))

    def add(self, value, priority=0):
        heappush(self.heap, (priority, value))

    def pop(self):
        priority, value = heappop(self.heap)
        return value

    def __len__(self):
        return len(self.heap)


def get_heuristic(h_fun, dim):
    def calc_h(cell):
        (i, j) = cell
        if h_fun == 'MANHATTAN':
            return abs(dim - i) + abs(dim - j)
        elif h_fun == 'EUCLIDEAN':
            return math.sqrt(abs(dim - i) ** 2 + abs(dim - j) ** 2)
        elif h_fun == 'CHEBYSHEV':
            return max(abs(dim - i), abs(dim - j))
        else:
            return max(abs(dim - i), abs(dim - j))
    return calc_h


def a_star_search(start, neighbors, heuristic, grid):
    dim = len(grid[0])
    visited = set()
    parent = dict()
    distance = {start: 0}
    fringe = PriorityQueue()
    fringe.add(start)

    while fringe:
        cell = fringe.pop()
        if cell in visited:
            continue
        if cell == (dim - (1 + padding), dim - (1 + padding)):
            return reconstruct_path(parent, start, cell)
        visited.add(cell)
        for child in neighbors(cell):
            fringe.add(child, priority=distance[cell] + 1 + heuristic(child))
            if child not in distance or distance[cell] + 1 < distance[child]:
                distance[child] = distance[cell] + 1
                parent[child] = cell
    return None


def reconstruct_path(parent, start, end):
    global x_train, y_train
    path = [end]
    while end != start:
        end = parent[end]
        path.append(end)
    return list(reversed(path))


def get_neighbors(grid, dim):
    def get_adjacent_cells(cell):
        x, y = cell
        return ((x + i, y + j)
                for (i, j) in [(1, 0), (0, 1), (0, -1), (-1, 0)]
                # (i, j) Represents movement from current cell - N,W,S,E direction eg: (1,0) means -> (x+1, y)
                # neighbor should be within grid boundary
                # neighbor should be an unblocked cell
                if 0 <= x + i < dim
                if 0 <= y + j < dim
                if grid[x + i][y + j] != 1)

    return get_adjacent_cells


def get_shortest_path(h_fun, grid):
    # Default start pos: (0,0)
    dim = len(grid[0])
    shortest_path = a_star_search(start, get_neighbors(grid, dim), get_heuristic(h_fun, dim), grid)
    if shortest_path is None:
        return -1
    else:
        return shortest_path

# Grid Representation

In [21]:
def draw_grid(img, grid_shape, color=(0, 255, 0), thickness=1):
    h, w, _ = img.shape
    rows, cols = grid_shape
    dy, dx = h / rows, w / cols

    # draw vertical lines
    for x in np.linspace(start=dx, stop=w-dx, num=cols-1):
        x = int(round(x))
        cv2.line(img, (x, 0), (x, h), color=color, thickness=thickness)

    # draw horizontal lines
    for y in np.linspace(start=dy, stop=h-dy, num=rows-1):
        y = int(round(y))
        cv2.line(img, (0, y), (w, y), color=color, thickness=thickness)

    return img

In [20]:
def draw(data,img2,visited):
    win_size=672
    cell_pixel=12 #  win_size/cell_pixel = no. of row+padding
    
    for i in range(0,win_size,cell_pixel):
        for j in range(0,win_size,cell_pixel):
            if(data[int(i/cell_pixel)][int(j/cell_pixel)]==0):
                img2=cv2.rectangle(img2,(i+1,j+1),(i+(cell_pixel-1),j+(cell_pixel-1)),(255,255,255),-1)
                
            if(data[int(i/cell_pixel)][int(j/cell_pixel)]==1):
                img2=cv2.rectangle(img2,(i+1,j+1),(i+(cell_pixel-1),j+(cell_pixel-1)),(0,0,0),-1)
                
            if(data[int(i/cell_pixel)][int(j/cell_pixel)]==2):
                img2=cv2.rectangle(img2,(i+1,j+1),(i+(cell_pixel-1),j+(cell_pixel-1)),(0,0,255),-1)
                
            if(data[int(i/cell_pixel)][int(j/cell_pixel)]==3):
                img2=cv2.rectangle(img2,(i+1,j+1),(i+(cell_pixel-1),j+(cell_pixel-1)),(0,255,0),-1)
            
    
  #  for (i,j) in visited:
    #    img2=cv2.rectangle(img2,((i*20)+1,(j*20)+1),(i*20+(cell_pixel-1),j*20+(cell_pixel-1)),(255,0,0),-1)


    return img2
        
        

# 

In [ ]:
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 1)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value

ct =1
folder_counter=1
success_grid_counter=1
solved=True
sucess=0
fail=0
while ct>0:
    num_moves=0
    visited=[]
    broke = 0
    p=0.3
    grid_dim = 50
    padding = 3
    x = np.pad(np.array(np.random.choice([0, 1], (grid_dim * grid_dim), p=[1 - p, p]).reshape(grid_dim, grid_dim)), padding, pad_with)
    dim = len(x[0])
    start = (padding, padding)
    x[start[0],start[1]] = 0
  #  print(dim - 1 - padding, dim - 1 - padding)
    x[dim - 1 - padding, dim - 1 - padding] = 0
    astrtime=time.time()
    a_star_path = get_shortest_path('MANHATTAN', x)
    if a_star_path!=-1:
        print("\n#########################################\n")
        print("Length: ",len(a_star_path))
        aendtime=time.time()
        print("A star solved in : ",aendtime-astrtime)
       
    else:
        broke = 1
       # print('Unsolvable')
        continue
   # print(x)
    nn_path = 0

    if a_star_path!=-1:
        img=np.ones((720,720,3))
        cv2.imwrite("grid"+str(success_grid_counter)+".jpg",draw(x,img,visited))
        
        inp = np.zeros((dim, dim))
        inp[start[0],start[1]] = 2
        inp[dim - 1 - padding, dim - 1 - padding] = 3
        seen = dict()
        for a in range(dim):
            for pad in range(0, padding):
                inp[a, pad] = 1
                inp[pad, a] = 1
                inp[dim - 1 - pad, a] = 1
                inp[a, dim - 1 - pad] = 1
        for coord in seen:
            inp[coord] = x[coord]
        (x1, y1) = start
        for (i, j) in [(1, 0), (0, 1), (0, -1), (-1, 0)]:
            if x1 + i == 0 or x1 + i == dim - 1 or y1 + i == 0 or y1 + i == dim - 1:
                pass
            else:
                inp[x1 + i][y1 + j] = x[x1 + i][y1 + j]
                seen[(x1 + i, y1 + j)] = x[x1 + i][y1 + j]




        c=0
        img=np.ones((720,720,3))
        img=draw_grid(img,(36,36),(0, 0, 0),1)
        path_buffer=[]
        import matplotlib.pyplot as plt
        if a_star_path!=-1:
            data = inp.copy()
            grid = data[x1 - padding:x1 + 1 + padding, y1 - padding:y1 + 1 + padding]

            a=cnn.predict(grid.reshape((1,7,7)))
            move=action[np.argmax(a)]
           # print(grid)
           # print(move)
            pos = start
            nn_path+=1
            strt_time=time.time()
            while move!='stop':
                visited.append(pos)
               # print(pos)
                num_moves+=1
                
                data[pos] = 0
                if move == 'left':
                    pos = (pos[0], pos[1]-1)
                elif move == 'right':
                    pos = (pos[0], pos[1]+1)
                elif move == 'up':
                    pos = (pos[0]-1, pos[1])
                elif move == 'down':
                    pos = (pos[0]+1, pos[1])
                else:
                    pos = pos

                (x1,y1)= pos
                for coord in seen:
                    data[coord] = x[coord]
                for (i, j) in [(1, 0), (0, 1), (0, -1), (-1, 0)]:
                    if (x1 + i == dim - padding and y1 + i == dim - padding) :
                        pass
                    else:
                        data[x1 + i][y1 + j] = x[x1 + i][y1 + j]
                        seen[(x1 + i, y1 + j)] = x[x1 + i][y1 + j]
                data[pos] = 2
                grid = data[x1 - padding :x1 + 1 + padding, y1 - padding:y1 + 1 + padding]
               # print(grid)
              #  print(data)
                img=draw(data,img,visited)
                path_buffer.append(pos)
                if len(path_buffer)>8:
                    path_buffer.pop(0)
               # print("Pth buffer: ",path_buffer)



                if(len(set(path_buffer))<4 and len(path_buffer)==8):
                    fail+=1
                    print("Breaking out since getting stuck in Infifinte loop\n")
                    success_grid_counter+=1
                #    os.remove("grid"+str(success_grid_counter)+".jpg")
                    for im in os.listdir("folder_"+str(folder_counter)):
                        os.remove("folder_"+str(folder_counter)+"/"+im)
                    broke = 1
                    break

    #             if(x[pos]==1):
    #                 print("ENtered Blocked cell")
    #                 continue




             #   plt.figure()
              #  plt.imshow(np.array(img,np.int32))
                if(not "folder_"+str(folder_counter) in os.listdir()):os.mkdir("folder_"+str(folder_counter))
                cv2.imwrite("folder_"+str(folder_counter)+"/"+"frmae"+str(c)+'.jpg',img)
                c+=1
              #  print(len(grid[0]))

                a=cnn.predict(grid.reshape((1,7,7)))
                move=action[np.argmax(a)]

               # print(move,"\n")
                nn_path+=1
    end_time=time.time()           
    if broke == 1:
        continue
    ct-=1
    print("Grid is Solved in: ",num_moves," time: ",end_time-strt_time)
    sucess+=1
    success_grid_counter+=1
    folder_counter+=1
    
#    print(nn_path)
#print(ct)